# Parsimonious flux balance analysis

We will use `parsimonious_flux_balance_analysis` and
`minimization_of_metabolic_adjustment` to find the optimal flux
distribution in the *E. coli* "core" model.

TODO pFBA citation

If it is not already present, download the model and load the package:

In [1]:
import Downloads: download

!isfile("e_coli_core.json") &&
    download("http://bigg.ucsd.edu/static/models/e_coli_core.json", "e_coli_core.json")

false

next, load the necessary packages

In [2]:
using COBREXA

import JSONFBCModels
import Clarabel # can solve QPs

model = load_model("e_coli_core.json") # load the model

JSONFBCModels.JSONFBCModel(#= 95 reactions, 72 metabolites =#)

Use the convenience function to run standard pFBA on

In [3]:
vt = parsimonious_flux_balance_analysis(model, Clarabel.Optimizer; settings = [silence])

ConstraintTrees.Tree{Float64} with 4 elements:
  :flux_stoichiometry     => ConstraintTrees.Tree{Float64}(#= 72 elements =#)
  :fluxes                 => ConstraintTrees.Tree{Float64}(#= 95 elements =#)
  :objective              => 0.873922
  :parsimonious_objective => 11414.2

# Alternatively, you can construct your own constraint tree model with
# the quadratic objective (this approach is much more flexible).

ctmodel = flux_balance_constraints(model)
ctmodel *= :l2objective^squared_sum_value(ctmodel.fluxes)
ctmodel.objective.bound = 0.3 # set growth rate # TODO currently breaks

opt_model = optimization_model(
    ctmodel;
    objective = ctmodel.:l2objective.value,
    optimizer = Clarabel.Optimizer,
    sense = Minimal,
)

J.optimize!(opt_model) # JuMP is called J in COBREXA

is_solved(opt_model) # check if solved

vt = C.substitute_values(ctmodel, J.value.(opt_model[:x])) # ConstraintTrees.jl is called C in COBREXA

@test isapprox(vt.l2objective, ?; atol = QP_TEST_TOLERANCE) #src  # TODO will break until mutable bounds

# It is likewise as simple to run MOMA using the convenience functions.

ref_sol = Dict("ATPS4r" => 33.0, "CYTBD" => 22.0)

vt = minimize_metabolic_adjustment(model, ref_sol, Gurobi.Optimizer)

# Or use the piping functionality

model |>
minimize_metabolic_adjustment(ref_sol, Clarabel.Optimizer; settings = [silence])


# Alternatively, you can construct your own constraint tree model with
# the quadratic objective (this approach is much more flexible).

ctmodel = flux_balance_constraints(model)
ctmodel *=
    :minoxphospho^squared_sum_error_value(
        ctmodel.fluxes,
        Dict(:ATPS4r => 33.0, :CYTBD => 22.0),
    )
ctmodel.objective.bound = 0.3 # set growth rate # TODO currently breaks

opt_model = optimization_model(
    ctmodel;
    objective = ctmodel.minoxphospho.value,
    optimizer = Clarabel.Optimizer,
    sense = Minimal,
)

J.optimize!(opt_model) # JuMP is called J in COBREXA

is_solved(opt_model) # check if solved

vt = C.substitute_values(ctmodel, J.value.(opt_model[:x])) # ConstraintTrees.jl is called C in COBREXA

@test isapprox(vt.l2objective, ?; atol = QP_TEST_TOLERANCE) #src  # TODO will break until mutable bounds

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*